# COMP90051 Project 2
**Name:** *enter your name here*

**Student ID:** *your id here*

In [31]:
# You can add additional imports here
import numpy as np
from matplotlib import pyplot as plt
import os
import random
from sklearn.model_selection import train_test_split
from collections import defaultdict
# Additional imports
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from numpy import std
import time


## 0. Loading the dataset

In [113]:
# do not edit this cell
# load the data files (download from the LMS)
embedded_images = np.load('images.npy')
labels = np.load('labels.npy')

# split into pool & testing
X_pool, X_test, y_pool, y_test = train_test_split(embedded_images, labels, 
                                                  test_size=0.5, random_state=1234, shuffle=True)

# sample a seed set
np.random.seed(1234)
label2id = defaultdict(list)
for i, label in enumerate(y_pool):
    label2id[label].append(i)
seed_set = []
for label, ids in label2id.items():
    seed_set.extend(np.random.choice(ids, size=10, replace=False))

In [52]:
#Helper Function
# 1 - Convert Ids to Labels of Seedset  
def id2label(seed_id,ListLabel):
    seed_set_label = []
    for label, ids in ListLabel.items():
        for seedid in seed_id:
            if seedid in ids:
                seed_set_label.append(label)  
    return seed_set_label
# 2 - Print Accuracy Score Of Model  
def printAccuracyScore(modelName,score):
    print(modelName, "Accuracy: {}".format(score))
    
    

## 1. Applying logistic regression

In [53]:
def train_logistic_regression(X, y, **args):
    """
    Train a logistic regression model on dataset (X, y) and return trained model.
    X: matrix of real values, size n x d
    y: vector of string labels, size n
    args: optional arguments e.g., for hyper-parameters
    """
    # your code here
    # run logistic regression model
    # NOTE = Find Best parameters  #Best: 0.302222 using {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
    start = time.time()
    
    log_reg = LogisticRegression(multi_class='multinomial',C=0.01,penalty='l2',solver='lbfgs')
    model = log_reg.fit(X, y)

    stop = time.time()
    print(f"Training time: {stop - start}s")

    return model


In [54]:
def evaluate_logistic_regression_accuracy(Xt, yt, model):
    """
    Apply logistic regression prediction on dataset Xt and evaluate accuracy against yt,
    returing the accuracy results as a scalar.
    Xt: matrix of real values, size m x d
    yt: vector of string labels, size m
    """
    # your code here
    y_pred = model.predict(Xt)
    score = accuracy_score(yt, y_pred)
    #print("Accuracy: {}".format(accuracy_score(yt, y_pred)))
    return score

  

In [60]:
# your code here for training, evaluating & plotting results

# Traning and Evaluation of Model (model_subset) with Subest of y_pool.
#seed_set_label = id2label(seed_set,label2id)
model_subset = train_logistic_regression(X_pool[seed_set],y_pool[seed_set])
score_model_subset = evaluate_logistic_regression_accuracy(X_test,y_test,model_subset)
#print("model_subset Accuracy: {}".format(score_model_subset))
printAccuracyScore("model_subset",score_model_subset)
print("\n")

# Traning and Evaluation of  Model (model_max_pool) with Max Pool 
model_max_pool = train_logistic_regression(X_pool,y_pool)
score_model_max_pool = evaluate_logistic_regression_accuracy(X_test,y_test,model_max_pool)
printAccuracyScore("model_max_pool",score_model_max_pool)
# Evaluation Report 




C:\Users\Waqar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
Training time: 0.7753183841705322s
model_subset Accuracy: 0.31732365145228214


Training time: 9.323550701141357s
model_max_pool Accuracy: 0.5675311203319502
C:\Users\Waqar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


## 2. Active learning framework with Random selection

In [56]:
def random_select(X, model, **args):
    """
    Given an unlabelled dataset X, a matrix of n x d, and a model (not used)
    returns a vector of scores of length n. Each entry reflects the priority 
    of the corresponding instance. Higher means better.
    """
    # Creating a score array using random numbers of length of X 
    scores = np.random.randint(1,len(X),len(X))
    return scores
    

In [117]:
def pool_based_active_learning(X_pool, y_pool, seed_ids,
                               train_func, select_func,
                               max_size, batch_size, **args):
    """
    Perform an active learning simulation, which starts by training on a seed set,
    then iteratively applies the selection function to rank instances in the pool,
    selects the top few instances which are included into the training set and the
    process repeats. 
        X_pool: matrix of n x d
        y_pool: vector of string labels, size n
        seed_ids: initial labelled set set, as a list of indices [0..n-1] into pool
        train_func: function which given (X, y, optional args) returns a trained model
        select_func: function which given (X, optional args) returns a sequence of scores
        max_size: stopping condition for active learning, when labelled data reaches given size
        batch_size: number of instances to be labelled in each iteration
        args: optional arguments passed to training and selection function
    returns the sequence of trained models 
    """
    performance_history=[]
    seed = seed_ids
    i = 0 
    # 1: U = pool of unlabelled instances, {x} 
    U =  np.delete(X_pool,seed, axis=0)
    # 2: L = set of initial labelled instances, {hx,yi} 
    L = [X_pool[seed],y_pool[seed]]   # L = (x,y)  
    # 3: b = number of instances to label in each step 
    b =  batch

    # 4: for t = 1,2,...,T do 
    #for i in range(1,46):  # Loop will run from 300 to 3000 Instances
    while(len(seed) < max_size):
        i+=1
    # 5:  θ(t) = train(L) 
        Theta_model=train_func(L[0],L[1])
   
    # 6:  score all instances in pool, r = select(U) 
        r = select_func(U,Theta_model)
    # 7:  for all j ∈ argmax(b,r) do 
        max_indices = np.argpartition(r,-b)[-b:]
        for j in max_indices:
    # 8:      reveal label y     
    # 9:      add hxj,yji to L
            L[0] = np.append(L[0],[X_pool[j]], axis=0)
            L[1]= np.append(L[1],y_pool[j])
            seed.append(j)
    # 10:     remove xj from U 
            U = np.delete(X_pool,seed,axis=0)
             #  Calculate and report our model's accuracy.                       
    # 11:    end for 
    # 12: end for 
    # 13: return {θ(t)}T t=1
 #  Calculate and report our model's accuracy.
        model_accuracy = evaluate_logistic_regression_accuracy(X_test,y_test,Theta_model)
        print('Accuracy after query {n}: {acc:0.4f}'.format(n=i, acc=model_accuracy))
    # Save our model's performance for plotting.
        performance_history.append(model_accuracy)
    
    return Theta_model
    ########################################## Not Part of Algo ########################
    
        
     #  Calculate and report our model's accuracy.
        #model_accuracy = evaluate_logistic_regression_accuracy(X_test,y_test,Theta_model)
        #print('Accuracy after query {n}: {acc:0.4f}'.format(n=i + 1, acc=model_accuracy))
    # # Save our model's performance for plotting.
        #performance_history.append(model_accuracy)

    ########################################## Not Part of Algo ########################


In [118]:
len(y_pool[seed_set])


3060

In [116]:
batch = 60
max_size = 3000
models_random = pool_based_active_learning(X_pool, y_pool, seed_set, 
                                    train_logistic_regression, random_select, 
                                    max_size, batch)

C:\Users\Waqar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
Training time: 0.8999893665313721s
Accuracy after query 1: 0.3173
C:\Users\Waqar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
Training time: 0.8380026817321777s
Accuracy after query 2: 0.3416
C:\Users\Waqar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
Training time: 0.8379945755004883s
Accuracy after query 3: 0.3451
C:\Users\Waqar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWar

In [91]:
len(seed_set)

2940

In [ ]:
## your code here for evaluation of accuracy and plotting of results

# NOTE: Make Graphs and Check Evaluation Scores

## 3. Uncertainty sampling

In [ ]:
def logistic_regression_entropy_select(X, model, **args):
    """
    Given an unlabelled dataset X, a matrix of n x d, and a discriminative model 
    P(y|x), returns a vector of n entropy values.
    """
    # fill in
    print("X Shape",X.shape)
    from scipy.stats import entropy
    #prob = model.predict_proba(X)   # paper has log 
    prob = model.predict_proba(X)
    #ar = prob.to_numpy()
    #print(ar.shape
    ent  = entropy(prob,axis=1,base=10)
    print("Ent shape ",ent.shape)
    print(ent)
    return ent
    #return ent.argmax()
    

In [ ]:
len(seed_set)
print(X_pool.shape)
print(y_pool.shape)

In [ ]:
models_us = pool_based_active_learning(X_pool, y_pool, seed_set, 
                                 train_logistic_regression, 
                                 logistic_regression_entropy_select, 
                                 max_size, batch)

In [ ]:
## your code here for evaluation of accuracy and plotting of results


## 4. Query by committee

In [ ]:
# don't forget to provide function descriptive comments, like those provided in templates above

def query_by_committee_vote_entropy(X, model, **args):
    pass

def query_by_committee_soft_vote_entropy(X, model, **args):
    pass

def query_by_committee_KL(X, model, **args):
    pass

In [ ]:
def train_committee(X, y, **args):
    pass

In [ ]:
## your code here for training, evaluation, and plotting code

## 5. Hierarchical sampling

In [ ]:
## your code